In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/MBX/quip_interface/test


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc","mbx_pbc_no_mww_pip","mbx_pbc_no_www_mww","mbx_pbc_no_ww_mw_www_mww",
              "mbx_pbc_no_ww_mw","mbx_pbc_no_ww","mbx_pbc_no_mw","mbx_pbc_no_ww_www_mww"]

In [9]:
structure_list=["1","2","3"]

In [10]:
dict_version_d_virial=OrderedDict([(struct,
                                    OrderedDict([(version,
                                        OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                       for version in version_list]))
                                    for struct in structure_list])

In [11]:
dict_version_model_virial=OrderedDict([(struct,
                                        OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                       for version in version_list]))
                                       for struct in structure_list]) 

In [12]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""
    pot.reset()

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [13]:
this_root="/home/es732/programs/MBX/quip_interface/test/"

In [14]:
this_struct="1"
this_file="phase_clathrate_2x2x2_"+this_struct+"_CH4.xyz"
this_at_tmp=quippy.Atoms(this_root+this_file)
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## diagonal_virial=F, only 2B mw PIP

In [15]:
this_version=version_list[7]
print this_version

mbx_pbc_no_ww_www_mww


In [16]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [17]:

at= this_at_tmp.copy()

In [18]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1130.27884335     2.23652556     4.3212633 ]
 [    2.23652556 -1147.06608404     5.36647072]
 [    4.3212633      5.36647072 -1142.2747285 ]]


In [19]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no PIPs

In [20]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_mw_www_mww


In [21]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [22]:
at = this_at_tmp.copy()

In [23]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1131.04480039     2.27380475     4.28009241]
 [    2.27380475 -1147.73769625     5.45378687]
 [    4.28009241     5.45378687 -1143.11236183]]


In [24]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [31]:
this_struct="2"
this_file="phase_clathrate_2x2x2_"+this_struct+"_CH4.xyz"
this_at_tmp=quippy.Atoms(this_root+this_file)
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## diagonal_virial=F, only 2B mw PIP

In [32]:
this_version=version_list[7]
print this_version

mbx_pbc_no_ww_www_mww


In [33]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [34]:

at= this_at_tmp.copy()

In [35]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1130.65329757     2.26553728     4.29187905]
 [    2.26553728 -1147.40649937     5.35703245]
 [    4.29187905     5.35703245 -1142.61638922]]


In [36]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no PIPs

In [37]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_mw_www_mww


In [38]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [39]:
at = this_at_tmp.copy()

In [40]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1132.10877804     2.27977331     4.29292089]
 [    2.27977331 -1148.77952016     5.4896803 ]
 [    4.29292089     5.4896803  -1144.25915268]]


In [41]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [42]:
this_struct="3"
this_file="phase_clathrate_2x2x2_"+this_struct+"_CH4.xyz"
this_at_tmp=quippy.Atoms(this_root+this_file)
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## diagonal_virial=F, only 2B mw PIP

In [43]:
this_version=version_list[7]
print this_version

mbx_pbc_no_ww_www_mww


In [44]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [45]:

at= this_at_tmp.copy()

In [46]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1130.95274017     2.26813386     4.31131634]
 [    2.26813386 -1147.80274147     5.34347151]
 [    4.31131634     5.34347151 -1142.97238631]]


In [47]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no PIPs

In [48]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_mw_www_mww


In [49]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [50]:
at = this_at_tmp.copy()

In [51]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1133.20991473     2.27502635     4.28738731]
 [    2.27502635 -1149.84385019     5.51453947]
 [    4.28738731     5.51453947 -1145.31327123]]


In [53]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## Errors in derivative virial / Virial from model (only diagonal)

## only 2B mw PIP

In [54]:
this_version1="mbx_pbc_no_ww_www_mww"
this_version2="mbx_pbc_no_ww_mw_www_mww"
for this_struct in ["1","2","3"]:
    print this_struct
    for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
        print "d =",tmp_d
        v1_numerical = dict_version_d_virial[this_struct][this_version1][tmp_d]
        v2_numerical = dict_version_d_virial[this_struct][this_version2][tmp_d]
        v1_model = dict_version_model_virial[this_struct][this_version1]
        v2_model = dict_version_model_virial[this_struct][this_version2]
        difference_numerical = v1_numerical-v2_numerical
        difference_model=v1_model-v2_model
        print np.diag((difference_numerical -difference_model)/difference_model)

1
d = 0.001
[-0.13863124 -0.16759354 -0.06655558]
d = 0.0001
[-0.13863696 -0.16758939 -0.06655517]
d = 1e-05
[-0.13863704 -0.16758948 -0.0665552 ]
d = 1e-06
[-0.13863699 -0.16758948 -0.06655508]
2
d = 0.001
[-0.14034233 -0.16247301 -0.06108817]
d = 0.0001
[-0.14035028 -0.16247457 -0.0610892 ]
d = 1e-05
[-0.14035037 -0.16247466 -0.06108923]
d = 1e-06
[-0.14035026 -0.16247467 -0.06108919]
3
d = 0.001
[-0.11492473 -0.1624846  -0.08284237]
d = 0.0001
[-0.11493125 -0.1624893  -0.08284559]
d = 1e-05
[-0.11493132 -0.1624894  -0.08284563]
d = 1e-06
[-0.11493127 -0.16248939 -0.08284557]
